In [2]:
from kfp.components import InputPath, OutputPath, create_component_from_func
from typing import Dict, List, NamedTuple, Optional, Union, Any

In [9]:
def run_evidently_report(
    input_data: InputPath('CSV') | InputPath('JSON'),
    output_path: OutputPath('HTML'),
    metrics: List[str] = ['data_quality'],
    cur_data: InputPath('CSV') | InputPath('JSON') = None,
    column_mapping: Optional[Dict[str, str]] = None
) -> None:
    
    import pandas as pd
    from pathlib import Path
    from evidently import ColumnMapping
    from evidently.report import Report
    from evidently.metric_preset import DataQualityPreset, DataDriftPreset, TargetDriftPreset ,RegressionPreset, ClassificationPreset
    
    PRESETS = {
        'data_quality': DataQualityPreset,
        'data_drift': DataDriftPreset,
        'target_drift': TargetDriftPreset,
        'regression': RegressionPreset,
        'classification': ClassificationPreset
    }
    
    try:
        ref_data = pd.read_csv(input_data)
    except Exception as e:
        raise ValueError(f"Error reading reference data: {e}")
    
    try:
        cur_data = pd.read_csv(cur_data)
    except Exception as e:
        cur_data=None
        raise ValueError(f"Error reading current data: {e}")
        
    
    report = Report(
        metrics=[PRESETS[preset]() for preset in metrics]
    )
    
    report.run(
        reference_data=ref_data,
        current_data=cur_data,
        column_mapping=column_mapping
    )
    
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    report.save_html(output_path)
    
    
    def _try_load_data(data: Any) -> pd.DataFrame:
        
        try:
            if isinstance(data, InputPath('CSV')):
                return pd.read_csv(data)
            else:
                return pd.read_json(data)
        except Exception as e:
            raise ValueError('Data should be a path to a csv file or a json object')
        
        
        
        
    
    
    
    
    